In [1]:
import pandas as pd
import numpy as np

DATA = 'data/'

In [2]:
test_df = pd.read_hdf(DATA + 'test_cleaned.h5', key='test')

test_df.shape
test_df.head()

(214200, 3)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [3]:
train_df = pd.read_hdf(DATA + 'train_augmented.h5', key='train')

In [4]:
items_df = pd.read_csv(DATA + 'items.csv')
shops_df = pd.read_csv(DATA + 'shops.csv')

items_df.head()
shops_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id,city_id
0,"!Якутск Орджоникидзе, 56 фран",0,1
1,"!Якутск ТЦ ""Центральный"" фран",1,1
2,"Адыгея ТЦ ""Мега""",2,2
3,"Балашиха ТРК ""Октябрь-Киномир""",3,3
4,"Волжский ТЦ ""Волга Молл""",4,4


In [5]:
test_df = test_df.merge(items_df, how='left', on='item_id')
test_df = test_df.merge(shops_df, how='left', on='shop_id')

test_df.drop('item_name', inplace=True, axis=1)
test_df.drop('shop_name', inplace=True, axis=1)

test_df['date_block_num'] = 34

In [6]:
test_df.head()

,ID,shop_id,item_id,item_category_id,city_id,date_block_num
0,0,5,5037,19,5,34
1,1,5,5320,55,5,34
2,2,5,5233,19,5,34
3,3,5,5232,23,5,34
4,4,5,5268,20,5,34


In [9]:
tmp = test_df.copy()

In [4]:
TIME_LAG = [1, 3, 6, 9, 12]

In [10]:
for i in TIME_LAG:
    tmp['lag%s' % i] = tmp['date_block_num'] - i

In [11]:
item_shop_sums_df = pd.read_csv(DATA + 'month_item_shop_sums.csv')

In [12]:
for i in TIME_LAG:
    left = ['lag%s' % i, 'item_id', 'shop_id']
    right = ['date_block_num', 'item_id', 'shop_id']
    tmp = tmp.merge(item_shop_sums_df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [16]:
test_df.shape
tmp.shape

tmp.head()

(214200, 6)

(214200, 16)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,lag12,item_lag_1,item_lag_3,item_lag_6,item_lag_9,item_lag_12
0,0,5,5037,19,5,34,33,31,28,25,22,0.0,3.0,1.0,0.0,1.0
1,1,5,5320,55,5,34,33,31,28,25,22,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,19,5,34,33,31,28,25,22,1.0,1.0,3.0,0.0,0.0
3,3,5,5232,23,5,34,33,31,28,25,22,0.0,1.0,0.0,0.0,0.0
4,4,5,5268,20,5,34,33,31,28,25,22,0.0,0.0,0.0,0.0,0.0


In [14]:
tmp.columns

Index(['ID', 'shop_id', 'item_id', 'item_category_id', 'city_id',
       'date_block_num', 'lag1', 'lag3', 'lag6', 'lag9', 'lag12',
       'date_block_num_1', 'item_cnt_day', 'date_block_num_3',
       'item_cnt_day_3', 'date_block_num_6', 'item_cnt_day_6',
       'date_block_num_9', 'item_cnt_day_9', 'date_block_num_12',
       'item_cnt_day_12'],
      dtype='object')

In [15]:
tmp.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

names = {"item_cnt_day_%s" % i: "item_lag_%s" % i for i in TIME_LAG}
names['item_cnt_day'] = 'item_lag_1'
names
tmp.rename(columns=names, inplace=True)

for i in TIME_LAG:
    tmp['item_lag_%s' % i].fillna(0, inplace=True)

{'item_cnt_day_1': 'item_lag_1',
 'item_cnt_day_3': 'item_lag_3',
 'item_cnt_day_6': 'item_lag_6',
 'item_cnt_day_9': 'item_lag_9',
 'item_cnt_day_12': 'item_lag_12',
 'item_cnt_day': 'item_lag_1'}

In [17]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [18]:
df = pd.read_csv(DATA + 'month_item_category_shop_sums.csv')

df.head()

,date_block_num,shop_id,item_category_id,item_cnt_day
0,0,2,0,0.0
1,0,2,2,10.0
2,0,2,3,0.0
3,0,2,5,2.0
4,0,2,6,10.0


In [19]:
for i in TIME_LAG:
    left = ['lag%s' % i, 'item_category_id', 'shop_id']
    right = ['date_block_num', 'item_category_id', 'shop_id']
    tmp = tmp.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [22]:
test_df.shape
tmp.shape

tmp.head()

(214200, 16)

(214200, 21)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_lag_1,item_lag_3,item_lag_6,item_lag_9,item_lag_12,item_category_lag_1,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12
0,0,5,5037,19,5,34,33,31,28,25,...,0.0,3.0,1.0,0.0,1.0,75.0,134.0,70.0,57.0,53.0
1,1,5,5320,55,5,34,33,31,28,25,...,0.0,0.0,0.0,0.0,0.0,104.0,136.0,97.0,103.0,104.0
2,2,5,5233,19,5,34,33,31,28,25,...,1.0,1.0,3.0,0.0,0.0,75.0,134.0,70.0,57.0,53.0
3,3,5,5232,23,5,34,33,31,28,25,...,0.0,1.0,0.0,0.0,0.0,54.0,73.0,43.0,32.0,60.0
4,4,5,5268,20,5,34,33,31,28,25,...,0.0,0.0,0.0,0.0,0.0,89.0,96.0,62.0,49.0,95.0


In [24]:
tmp.drop(['date_block_num_%s' % i for i in TIME_LAG],
         inplace=True,
         axis=1)

names = {
    "item_cnt_day_%s" % i: "item_category_lag_%s" % i
    for i in TIME_LAG
}
names['item_cnt_day'] = 'item_category_lag_1'
names
tmp.rename(columns=names, inplace=True)

for i in TIME_LAG:
    tmp['item_category_lag_%s' % i].fillna(0, inplace=True)

In [25]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [26]:
df = pd.read_csv(DATA + 'item_shop_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0


In [27]:
df = train_df.join(df)

In [28]:
train_df[(train_df.shop_id == 59) & (train_df.item_id == 22154)]

,date_block_num,item_price,item_cnt_day,shop_id,item_id


In [29]:
df.shape
df.head()

(7282800, 7)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,cumsum,cumcount
0,0,1999.0,0.0,5,5037,0.0,0
1,0,999.0,0.0,50,19034,0.0,0
2,0,799.0,0.0,35,15451,0.0,0
3,0,149.0,0.0,50,18968,0.0,0
4,0,299.0,0.0,50,18047,0.0,0


In [37]:
df['shop_id'] = df.shop_id.astype(np.int32)
df['item_id'] = df.item_id.astype(np.int32)

train_df['shop_id'] = train_df.shop_id.astype(np.int32)
train_df['item_id'] = train_df.item_id.astype(np.int32)

In [39]:
df.drop(['date_block_num', 'item_price', 'item_cnt_day'], inplace=True, axis=1)
df.drop_duplicates(subset=['shop_id', 'item_id'], keep='last', inplace=True)

tmp = tmp.merge(df, how='left', on=['shop_id', 'item_id'])
tmp.rename(columns={'cumsum': 'item_cumsum', 'cumcount': 'item_cumcount'}, inplace=True)

In [44]:
test_df.shape
tmp.shape

tmp.head()

(214200, 21)

(214200, 24)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_lag_9,item_lag_12,item_category_lag_1,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg
0,0,5,5037,19,5,34,33,31,28,25,...,0.0,1.0,75.0,134.0,70.0,57.0,53.0,3.0,6,0.5
1,1,5,5320,55,5,34,33,31,28,25,...,0.0,0.0,104.0,136.0,97.0,103.0,104.0,0.0,6,0.0
2,2,5,5233,19,5,34,33,31,28,25,...,0.0,0.0,75.0,134.0,70.0,57.0,53.0,0.0,6,0.0
3,3,5,5232,23,5,34,33,31,28,25,...,0.0,0.0,54.0,73.0,43.0,32.0,60.0,0.0,6,0.0
4,4,5,5268,20,5,34,33,31,28,25,...,0.0,0.0,89.0,96.0,62.0,49.0,95.0,0.0,6,0.0


In [41]:
tmp[tmp.item_cumsum.isna()].shape

(0, 23)

In [42]:
tmp.item_cumcount.fillna(0, inplace=True)
tmp.item_cumsum.fillna(0, inplace=True)

In [43]:
mask = tmp.item_cumcount != 0
tmp['item_avg'] = 0
tmp.loc[mask, 'item_avg'] = tmp[mask].item_cumsum / tmp[mask].item_cumcount

In [45]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [46]:
df = pd.read_csv(DATA + 'item_category_shop_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,1
3,0.0,2
4,0.0,3


In [47]:
df = train_df.join(df)
df = df.merge(items_df, how='left', on='item_id')

In [48]:
df.shape
df.head()

(7282800, 9)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,cumsum,cumcount,item_name,item_category_id
0,0,1999.0,0.0,5,5037,0.0,0,"NHL 15 [PS3, русские субтитры]",19
1,0,999.0,0.0,50,19034,0.0,0,СПИСОК ШИНДЛЕРА (BD+DVD с доп. материалами+бук...,37
2,0,799.0,0.0,35,15451,0.0,1,Мягкая игрушка СОЮЗМУЛЬТФИЛЬМ Пес Шарик музыка...,63
3,0,149.0,0.0,50,18968,0.0,2,СОКРОВИЩЕ НАЦИИ: КНИГА ТАЙН (регион),40
4,0,299.0,0.0,50,18047,0.0,3,Резинки для плетения силиконовые Мармеладный м...,70


In [49]:
df.drop(
    ['date_block_num', 'item_id', 'item_price', 'item_cnt_day', 'item_name'],
    inplace=True,
    axis=1)

In [50]:
df.drop_duplicates(subset=['shop_id', 'item_category_id'], keep='last', inplace=True)
tmp = tmp.merge(df, how='left', on=['shop_id', 'item_category_id'])

In [51]:
tmp.shape
tmp.head()

(214200, 26)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_category_lag_1,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg,cumsum,cumcount
0,0,5,5037,19,5,34,33,31,28,25,...,75.0,134.0,70.0,57.0,53.0,3.0,6,0.5,1425.0,5541
1,1,5,5320,55,5,34,33,31,28,25,...,104.0,136.0,97.0,103.0,104.0,0.0,6,0.0,102.0,666
2,2,5,5233,19,5,34,33,31,28,25,...,75.0,134.0,70.0,57.0,53.0,0.0,6,0.0,1425.0,5541
3,3,5,5232,23,5,34,33,31,28,25,...,54.0,73.0,43.0,32.0,60.0,0.0,6,0.0,0.0,1092
4,4,5,5268,20,5,34,33,31,28,25,...,89.0,96.0,62.0,49.0,95.0,0.0,6,0.0,679.0,1028


In [52]:
tmp.rename(columns={'cumsum': 'item_category_cumsum', 'cumcount': 'item_category_cumcount'}, inplace=True)

In [53]:
tmp[tmp.item_category_cumsum.isna()].shape

(0, 26)

In [54]:
tmp.item_category_cumsum.fillna(0, inplace=True)
tmp.item_category_cumcount.fillna(0, inplace=True)

In [55]:
mask = tmp.item_category_cumcount != 0
tmp['item_category_avg'] = 0
tmp.loc[mask, 'item_category_avg'] = tmp[mask].item_category_cumsum / tmp[mask].item_category_cumcount

In [56]:
test_df.shape
tmp.shape

tmp.head()

(214200, 24)

(214200, 27)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg,item_category_cumsum,item_category_cumcount,item_category_avg
0,0,5,5037,19,5,34,33,31,28,25,...,134.0,70.0,57.0,53.0,3.0,6,0.5,1425.0,5541,0.257174
1,1,5,5320,55,5,34,33,31,28,25,...,136.0,97.0,103.0,104.0,0.0,6,0.0,102.0,666,0.153153
2,2,5,5233,19,5,34,33,31,28,25,...,134.0,70.0,57.0,53.0,0.0,6,0.0,1425.0,5541,0.257174
3,3,5,5232,23,5,34,33,31,28,25,...,73.0,43.0,32.0,60.0,0.0,6,0.0,0.0,1092,0.000000
4,4,5,5268,20,5,34,33,31,28,25,...,96.0,62.0,49.0,95.0,0.0,6,0.0,679.0,1028,0.660506


In [57]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [58]:
df = pd.read_csv(DATA + 'last_prices.csv', index_col=0)
df.head()

,item_id,item_price
7277700,16315,949.0
7277701,15571,229.0
7277702,19657,730.0
7277703,7223,3999.0
7277704,7215,1199.0


In [59]:
tmp = tmp.merge(df, how='left', on='item_id')

In [60]:
tmp.shape
tmp.head()

(214200, 28)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_category_lag_6,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg,item_category_cumsum,item_category_cumcount,item_category_avg,item_price
0,0,5,5037,19,5,34,33,31,28,25,...,70.0,57.0,53.0,3.0,6,0.5,1425.0,5541,0.257174,1499.0
1,1,5,5320,55,5,34,33,31,28,25,...,97.0,103.0,104.0,0.0,6,0.0,102.0,666,0.153153,NaN
2,2,5,5233,19,5,34,33,31,28,25,...,70.0,57.0,53.0,0.0,6,0.0,1425.0,5541,0.257174,1199.0
3,3,5,5232,23,5,34,33,31,28,25,...,43.0,32.0,60.0,0.0,6,0.0,0.0,1092,0.000000,1199.0
4,4,5,5268,20,5,34,33,31,28,25,...,62.0,49.0,95.0,0.0,6,0.0,679.0,1028,0.660506,NaN


In [61]:
tmp[tmp.item_price.isna()].shape
tmp[tmp.item_price == 0].shape

(16128, 28)

(0, 28)

In [63]:
grouped = train_df.groupby('item_id').item_price.median()
grouped = grouped.to_frame().reset_index()

In [69]:
tmp = tmp.merge(grouped, how='left', on='item_id')

In [70]:
tmp.head()

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg,item_category_cumsum,item_category_cumcount,item_category_avg,item_price_x,item_price_y
0,0,5,5037,19,5,34,33,31,28,25,...,57.0,53.0,3.0,6,0.5,1425.0,5541,0.257174,1499.0,1999.0
1,1,5,5320,55,5,34,33,31,28,25,...,103.0,104.0,0.0,6,0.0,102.0,666,0.153153,NaN,NaN
2,2,5,5233,19,5,34,33,31,28,25,...,57.0,53.0,0.0,6,0.0,1425.0,5541,0.257174,1199.0,599.5
3,3,5,5232,23,5,34,33,31,28,25,...,32.0,60.0,0.0,6,0.0,0.0,1092,0.000000,1199.0,599.5
4,4,5,5268,20,5,34,33,31,28,25,...,49.0,95.0,0.0,6,0.0,679.0,1028,0.660506,NaN,NaN


In [80]:
tmp.drop('item_price_y', inplace=True, axis=1)
tmp.rename({'item_price_x': 'item_price'}, inplace=True, axis=1)

In [78]:
# tmp.item_price.fillna(0, inplace=True)

In [81]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [82]:
df = pd.read_csv(DATA + 'month_item_city_sums.csv')

df.shape
df.head()

(4681800, 4)

,date_block_num,city_id,item_id,item_cnt_day
0,0,1,30,0.0
1,0,1,31,0.0
2,0,1,32,13.0
3,0,1,33,3.0
4,0,1,38,0.0


In [84]:
tmp = test_df.copy()

for i in TIME_LAG:
    left = ['lag%s' % i, 'item_id', 'city_id']
    right = ['date_block_num', 'item_id', 'city_id']
    tmp = tmp.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [87]:
test_df.shape
tmp.shape

tmp.head()

(214200, 28)

(214200, 33)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_avg,item_category_cumsum,item_category_cumcount,item_category_avg,item_price,item_city_lag_1,item_city_lag_3,item_city_lag_6,item_city_lag_9,item_city_lag_12
0,0,5,5037,19,5,34,33,31,28,25,...,0.5,1425.0,5541,0.257174,1499.0,0.0,3.0,1.0,0.0,1.0
1,1,5,5320,55,5,34,33,31,28,25,...,0.0,102.0,666,0.153153,NaN,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,19,5,34,33,31,28,25,...,0.0,1425.0,5541,0.257174,1199.0,1.0,1.0,3.0,0.0,0.0
3,3,5,5232,23,5,34,33,31,28,25,...,0.0,0.0,1092,0.000000,1199.0,0.0,1.0,0.0,0.0,0.0
4,4,5,5268,20,5,34,33,31,28,25,...,0.0,679.0,1028,0.660506,NaN,0.0,0.0,0.0,0.0,0.0


In [86]:
tmp.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

names = {
    "item_cnt_day_%s" % i: "item_city_lag_%s" % i
    for i in TIME_LAG
}
names['item_cnt_day'] = 'item_city_lag_1'
names
tmp.rename(columns=names, inplace=True)

for i in TIME_LAG:
    tmp['item_city_lag_%s' % i].fillna(0, inplace=True)

{'item_cnt_day_1': 'item_city_lag_1',
 'item_cnt_day_3': 'item_city_lag_3',
 'item_cnt_day_6': 'item_city_lag_6',
 'item_cnt_day_9': 'item_city_lag_9',
 'item_cnt_day_12': 'item_city_lag_12',
 'item_cnt_day': 'item_city_lag_1'}

In [88]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [89]:
df = pd.read_csv(DATA + 'month_item_category_city_sums.csv')

df.shape
df.head()

(56916, 4)

,date_block_num,city_id,item_category_id,item_cnt_day
0,0,1,0,0.0
1,0,1,2,32.0
2,0,1,3,0.0
3,0,1,5,7.0
4,0,1,6,67.0


In [90]:
tmp = test_df.copy()

for i in TIME_LAG:
    left = ['lag%s' % i, 'item_category_id', 'city_id']
    right = ['date_block_num', 'item_category_id', 'city_id']
    tmp = tmp.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [93]:
test_df.shape
tmp.shape

tmp.head()

(214200, 33)

(214200, 38)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_city_lag_1,item_city_lag_3,item_city_lag_6,item_city_lag_9,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12
0,0,5,5037,19,5,34,33,31,28,25,...,0.0,3.0,1.0,0.0,1.0,75.0,134.0,70.0,57.0,53.0
1,1,5,5320,55,5,34,33,31,28,25,...,0.0,0.0,0.0,0.0,0.0,104.0,136.0,97.0,103.0,104.0
2,2,5,5233,19,5,34,33,31,28,25,...,1.0,1.0,3.0,0.0,0.0,75.0,134.0,70.0,57.0,53.0
3,3,5,5232,23,5,34,33,31,28,25,...,0.0,1.0,0.0,0.0,0.0,54.0,73.0,43.0,32.0,60.0
4,4,5,5268,20,5,34,33,31,28,25,...,0.0,0.0,0.0,0.0,0.0,89.0,96.0,62.0,49.0,95.0


In [92]:
tmp.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

names = {
    "item_cnt_day_%s" % i: "item_category_city_lag_%s" % i
    for i in TIME_LAG
}
names['item_cnt_day'] = 'item_category_city_lag_1'
names
tmp.rename(columns=names, inplace=True)

for i in TIME_LAG:
    tmp['item_category_city_lag_%s' % i].fillna(0, inplace=True)

{'item_cnt_day_1': 'item_category_city_lag_1',
 'item_cnt_day_3': 'item_category_city_lag_3',
 'item_cnt_day_6': 'item_category_city_lag_6',
 'item_cnt_day_9': 'item_category_city_lag_9',
 'item_cnt_day_12': 'item_category_city_lag_12',
 'item_cnt_day': 'item_category_city_lag_1'}

In [94]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [95]:
df = pd.read_csv(DATA + 'item_city_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0


In [96]:
df = train_df.join(df)
df = df.merge(shops_df, how='left', on='shop_id')

In [97]:
df.shape
df.head()

(7282800, 9)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,cumsum,cumcount,shop_name,city_id
0,0,1999.0,0.0,5,5037,0.0,0,"Вологда ТРЦ ""Мармелад""",5
1,0,999.0,0.0,50,19034,0.0,0,"Тюмень ТЦ ""Гудвин""",25
2,0,799.0,0.0,35,15451,0.0,0,"Н.Новгород ТРЦ ""Фантастика""",16
3,0,149.0,0.0,50,18968,0.0,0,"Тюмень ТЦ ""Гудвин""",25
4,0,299.0,0.0,50,18047,0.0,0,"Тюмень ТЦ ""Гудвин""",25


In [100]:
df.drop(['date_block_num', 'item_price', 'item_cnt_day', 'shop_name', 'shop_id'], inplace=True, axis=1)
df.drop_duplicates(subset=['city_id', 'item_id'], keep='last', inplace=True)

tmp = test_df.merge(df, how='left', on=['city_id', 'item_id'])
tmp.rename(columns={'cumsum': 'item_city_cumsum', 'cumcount': 'item_city_cumcount'}, inplace=True)

tmp[tmp.item_city_cumsum.isna()].shape

tmp.item_city_cumsum.fillna(0, inplace=True)
tmp.item_city_cumcount.fillna(0, inplace=True)

(0, 40)

In [102]:
mask = tmp.item_city_cumcount != 0
tmp['item_city_avg'] = 0
tmp.loc[mask, 'item_city_avg'] = tmp[mask].item_city_cumsum / tmp[mask].item_city_cumcount

In [103]:
test_df.shape
tmp.shape

tmp.head()

(214200, 38)

(214200, 41)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_city_lag_9,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg
0,0,5,5037,19,5,34,33,31,28,25,...,0.0,1.0,75.0,134.0,70.0,57.0,53.0,69.0,210,0.328571
1,1,5,5320,55,5,34,33,31,28,25,...,0.0,0.0,104.0,136.0,97.0,103.0,104.0,0.0,210,0.000000
2,2,5,5233,19,5,34,33,31,28,25,...,0.0,0.0,75.0,134.0,70.0,57.0,53.0,0.0,210,0.000000
3,3,5,5232,23,5,34,33,31,28,25,...,0.0,0.0,54.0,73.0,43.0,32.0,60.0,0.0,210,0.000000
4,4,5,5268,20,5,34,33,31,28,25,...,0.0,0.0,89.0,96.0,62.0,49.0,95.0,23.0,210,0.109524


In [104]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [105]:
df = pd.read_csv(DATA + 'item_category_city_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,1
3,0.0,2
4,0.0,3


In [106]:
df = train_df.join(df)
df = df.merge(shops_df, how='left', on='shop_id')
df = df.merge(items_df, how='left', on='item_id')

In [107]:
df.shape
df.head()

(7282800, 11)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,cumsum,cumcount,shop_name,city_id,item_name,item_category_id
0,0,1999.0,0.0,5,5037,0.0,0,"Вологда ТРЦ ""Мармелад""",5,"NHL 15 [PS3, русские субтитры]",19
1,0,999.0,0.0,50,19034,0.0,0,"Тюмень ТЦ ""Гудвин""",25,СПИСОК ШИНДЛЕРА (BD+DVD с доп. материалами+бук...,37
2,0,799.0,0.0,35,15451,0.0,1,"Н.Новгород ТРЦ ""Фантастика""",16,Мягкая игрушка СОЮЗМУЛЬТФИЛЬМ Пес Шарик музыка...,63
3,0,149.0,0.0,50,18968,0.0,2,"Тюмень ТЦ ""Гудвин""",25,СОКРОВИЩЕ НАЦИИ: КНИГА ТАЙН (регион),40
4,0,299.0,0.0,50,18047,0.0,3,"Тюмень ТЦ ""Гудвин""",25,Резинки для плетения силиконовые Мармеладный м...,70


In [108]:
cols_to_delete = [
    'date_block_num', 'shop_id', 'item_price', 'item_cnt_day', 'shop_name',
    'item_name', 'item_id'
]
df.drop(cols_to_delete, inplace=True, axis=1)
df.drop_duplicates(
    subset=['city_id', 'item_category_id'], keep='last', inplace=True)

tmp = test_df.merge(df, how='left', on=['city_id', 'item_category_id'])
tmp.rename(
    columns={
        'cumsum': 'item_category_city_cumsum',
        'cumcount': 'item_category_city_cumcount'
    },
    inplace=True)

tmp[tmp.item_city_cumsum.isna()].shape

tmp.item_category_city_cumsum.fillna(0, inplace=True)
tmp.item_category_city_cumcount.fillna(0, inplace=True)

(0, 43)

In [109]:
mask = tmp.item_category_city_cumcount != 0
tmp['item_category_city_avg'] = 0
tmp.loc[mask, 'item_category_city_avg'] = tmp[mask].item_category_city_cumsum / tmp[mask].item_category_city_cumcount

In [110]:
test_df.shape
tmp.shape

tmp.head()

(214200, 41)

(214200, 44)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,lag1,lag3,lag6,lag9,...,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg
0,0,5,5037,19,5,34,33,31,28,25,...,134.0,70.0,57.0,53.0,69.0,210,0.328571,45246.0,162621,0.278230
1,1,5,5320,55,5,34,33,31,28,25,...,136.0,97.0,103.0,104.0,0.0,210,0.000000,2584.0,19842,0.130229
2,2,5,5233,19,5,34,33,31,28,25,...,134.0,70.0,57.0,53.0,0.0,210,0.000000,45246.0,162621,0.278230
3,3,5,5232,23,5,34,33,31,28,25,...,73.0,43.0,32.0,60.0,0.0,210,0.000000,1381.0,31896,0.043297
4,4,5,5268,20,5,34,33,31,28,25,...,96.0,62.0,49.0,95.0,23.0,210,0.109524,20848.0,29588,0.704610


In [111]:
test_df = tmp.copy()
test_df.to_csv(DATA + 'features_test.csv', index=False)

In [112]:
tmp.columns

Index(['ID', 'shop_id', 'item_id', 'item_category_id', 'city_id',
       'date_block_num', 'lag1', 'lag3', 'lag6', 'lag9', 'lag12', 'item_lag_1',
       'item_lag_3', 'item_lag_6', 'item_lag_9', 'item_lag_12',
       'item_category_lag_1', 'item_category_lag_3', 'item_category_lag_6',
       'item_category_lag_9', 'item_category_lag_12', 'item_cumsum',
       'item_cumcount', 'item_avg', 'item_category_cumsum',
       'item_category_cumcount', 'item_category_avg', 'item_price',
       'item_city_lag_1', 'item_city_lag_3', 'item_city_lag_6',
       'item_city_lag_9', 'item_city_lag_12', 'item_category_city_lag_1',
       'item_category_city_lag_3', 'item_category_city_lag_6',
       'item_category_city_lag_9', 'item_category_city_lag_12',
       'item_city_cumsum', 'item_city_cumcount', 'item_city_avg',
       'item_category_city_cumsum', 'item_category_city_cumcount',
       'item_category_city_avg'],
      dtype='object')

In [113]:
tmp.drop(['lag%s' % i for i in TIME_LAG], inplace=True, axis=1)

In [114]:
tmp.to_hdf(DATA + 'features_test.h5', key='data', format='table', mode='w')

In [8]:
feature_df = pd.read_hdf(DATA + 'features_test.h5', key='data')

In [9]:
feature_df.shape

feature_df.head()

(214200, 39)

,ID,shop_id,item_id,item_category_id,city_id,date_block_num,item_lag_1,item_lag_3,item_lag_6,item_lag_9,...,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg
0,0,5,5037,19,5,34,0.0,3.0,1.0,0.0,...,134.0,70.0,57.0,53.0,69.0,210,0.328571,45246.0,162621,0.278230
1,1,5,5320,55,5,34,0.0,0.0,0.0,0.0,...,136.0,97.0,103.0,104.0,0.0,210,0.000000,2584.0,19842,0.130229
2,2,5,5233,19,5,34,1.0,1.0,3.0,0.0,...,134.0,70.0,57.0,53.0,0.0,210,0.000000,45246.0,162621,0.278230
3,3,5,5232,23,5,34,0.0,1.0,0.0,0.0,...,73.0,43.0,32.0,60.0,0.0,210,0.000000,1381.0,31896,0.043297
4,4,5,5268,20,5,34,0.0,0.0,0.0,0.0,...,96.0,62.0,49.0,95.0,23.0,210,0.109524,20848.0,29588,0.704610


In [14]:
feature_df['month'] = (feature_df.date_block_num % 12) + 1
days_per_month = pd.Series([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], index=range(1, 13), name='days')

feature_df = feature_df.merge(days_per_month.to_frame(), how="left", left_on='month', right_index=True)

In [10]:
templates = ['item_lag_%s', 'item_category_lag_%s', 'item_city_lag_%s', 'item_category_city_lag_%s']
columns = [t % i for t in templates for i in TIME_LAG]

In [11]:
feature_df.drop([c for c in feature_df.columns if c.endswith('_cumsum') or c.endswith('_cumcount')], inplace=True, axis=1)

In [15]:
for c in columns:
    feature_df[c] = feature_df[c] / feature_df.days

In [16]:
feature_df.to_hdf(DATA + 'features_test_v1.h5', key='features', format='table', mode='w')